In [2]:
import os
os.chdir('../../')

In [3]:
os.getcwd()

'/mnt/weka/home/zhihan.yang/bd3lms'

In [4]:
import os
from hydra import compose, initialize
from omegaconf import OmegaConf
import torch
import json
from diffusion import Diffusion
from tqdm import tqdm
from metrics import Metrics

from dataloader import get_dataloaders, get_tokenizer
from diffusion import Diffusion
from utils import get_logger

In [ ]:
# Register resolvers
OmegaConf.register_new_resolver('device_count', lambda: torch.cuda.device_count())
OmegaConf.register_new_resolver('eval', lambda x: eval(x))
OmegaConf.register_new_resolver('div_up', lambda x, y: (x + y - 1) // y)

In [25]:
# works for all samples on OWT (assuming a generation length of 1024)

def compute_metrics(samples_dir):

    print(samples_dir)

    with initialize(version_base=None, config_path="../../configs"):
        config = compose(
            config_name="config", 
            overrides=[
                "data=openwebtext-split"])  # required to get the right tokenizer

    # during init, Metrics set the following two important attributes:
    # - self.eval_ppl_batch_size as config.eval.perplexity_batch_size (8 by default)
    # - self.gen_ppl_eval_model_name_or_path as config.eval.gen_ppl_eval_model_name_or_path (gpt2-large by default)
    # nothing else matters

    metrics = Metrics(config=config)

    samples = []
    for fname in os.listdir(samples_dir):
        fpath = os.path.join(samples_dir, fname)
        with open(fpath, 'r') as f:
            loaded_dict = json.load(f)
        samples.extend(loaded_dict['samples'])

    print(f"Loaded {len(samples)} samples")

    # gen ppl
    
    metrics.reset()
    metrics.record_generative_perplexity(
        samples,
        config.model.length,
        config.loader.eval_batch_size,  # overwrites self.eval_ppl_batch_size
        retokenize=True)  
    gen_ppl = metrics.gen_ppl.compute().cpu().item()
    print(f"Generative perplexity: {gen_ppl:.2f}")

    return
    
    # entropy

    tokenizer = None

    entropies = []
    for sample in samples:
        token_ids = torch.tensor(tokenizer.encode(sample, add_special_tokens=False))
        counts = torch.unique(token_ids, return_counts=True)[1]
        entropy = torch.special.entr(counts.float() / counts.sum()).sum().item()
        entropies.append(entropy)
    avg_entropy = sum(entropies) / len(entropies)
    print(f"Average entropy: {avg_entropy:.2f}")

    # mauve

    # TODO

    # save results
    
    results = {
        "gen_ppl": gen_ppl,
        "entropy": avg_entropy,
        "avg_nfe": avg_nfe,
        "mauve": mauve_score
    }
    output_metrics_file = os.path.join(output_dir, f"_res_metrics_nucleus_{p}_t_{t}_numtries_{num_tries_val}_block_{block_size}_firsthit_{first_hit_val}.json")
    
    # TODO

    with open(output_metrics_file, "w") as f:
        json.dump(results, f, indent=4)
    print(f"Saved metrics to {output_metrics_file}")

In [26]:
samples_dir = "/mnt/weka/home/zhihan.yang/samples/mdlm/blocksize_1024_tval_4_numtries_1_nucleus_0.9_firsthit_false"

In [27]:
samples = compute_metrics(samples_dir)

/mnt/weka/home/zhihan.yang/samples/mdlm/blocksize_1024_tval_4_numtries_1_nucleus_0.9_firsthit_false


Loaded 1000 samples


/mnt/weka/home/zhihan.yang/miniconda3/envs/bd3lm/lib/python3.9/site-packages/torch/cuda/__init__.py:789: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


InterpolationResolutionError: ZeroDivisionError raised while resolving interpolation: integer division or modulo by zero
    full_key: loader.eval_batch_size
    object_type=dict